In [25]:
#Import functions
import polars as pl
from functools import partial
from itertools import product
import multiprocessing
from numba import jit
from collections import Counter
import numpy as np

In [26]:
#Get dataframe of all tickers
constituents = pl.read_csv("../Data/SP500/Constituents.csv", infer_schema_length=False)

#Create list of individual stocks from names from dataframe
stockComponents=[]
for k in range(len(constituents)):
    stockComponents.append(constituents[k,0])
    #stockComponents.append(constituents[k,0]+'.L')

In [27]:
#Load prices and timestamps from CSVs, just AAPL for now, TSLA soon
currentDf=pl.read_csv("../Data/SP500/minuteHist2021/tradingHours/AAPL.csv", infer_schema_length=None)
openPrices=currentDf['open'].to_numpy()
closePrices=currentDf['close'].to_numpy()
timestamps=currentDf['time'].to_numpy()

In [30]:
#Function for sma
#Using numba for speed
@jit(nopython=True)
def sma(length, date, dataframe):
    start=date-length
    end=date

    total = 0
    #Simple mean by add and divide
    for k in range(start,end+1):
        total+=int(dataframe[k])
    average = total/length

    return (average)

In [31]:
#Function for ema
#Using numba for speed
@jit(nopython=True)
def ema(length, date, dataframe):
    start=date-length
    end=date

    total = 0
    
    #Exponential average by using equation of (1/2 * 1/2^n * price) (a/1-r) (1/3 * 2/3^n * price) which = 1 as it tends to infinity
    #How much emphasis on recent data vs old?

    expoError=0
    divisor=4

    j=0
    for k in range(start+1,end+1):
        total+=(int(dataframe[k]) * (1/divisor * (1-(1/divisor))**j))
        j+=1

    j=0
    for k in range(start+1,end+1):
        expoError+=(1/divisor * (1-(1/divisor))**j)
        j+=1

    average = total/expoError
    #print(total, expoError)

    return average

In [46]:
#Algo
#Numba for speed
@jit(nopython=True)
#Takes in list of paramaters and 3 separate arrays, open, close, time
def algo(paramsList, openPrices, closePrices, timestamps):

    change=paramsList[0] #Percent change in price to look for
    takeProfit=paramsList[1] #The take profit percentage
    stopLoss=paramsList[2] #The stop loss percentage
    timeout=paramsList[3] #The timeout when it takes too long
    fee=paramsList[4] #Fee on top of every transaction
    ticksBefore=paramsList[5] #Price point beforehand to look at
    version=paramsList[6]
    
    tradePercent=0
    totalReturn=1

    total=0

    start=60 #Start of data
    end=int(len(openPrices)-61) #End of data

    for j in range(start, end):

        #Check that its not within the first or last hour of trading
        if timestamps[j]-timestamps[j-60]<4500000 and timestamps[j+60]-timestamps[j]<4500000:

            #Set start price
            startPrice=openPrices[j]

            #Set price we are checking
            if version==1:
                checkPrice=openPrices[j-ticksBefore]


            elif version==2:
                #checkPrice=ema(ticksBefore, j, openPrices)
                

                checkTotal = 0
                #Exponential average by using equation of (1/2 * 1/2^n * price) (a/1-r) (1/3 * 2/3^n * price) which = 1 as it tends to infinity
                expoError=0
                divisor=4

                count=0
                for k in range(j-ticksBefore+1,j+1):
                    checkTotal+=(int(openPrices[k]) * (1/divisor * (1-(1/divisor))**(ticksBefore-count)))
                    count+=1

                count=0
                for k in range(j-ticksBefore+1,j+1):
                    expoError+=(1/divisor * (1-(1/divisor))**(ticksBefore-count))
                    count+=1

                checkPrice = checkTotal/expoError


            elif version==3:
                #checkPrice=sma(ticksBefore, j, openPrices)
                checkTotal = 0
                #Simple mean by add and divide
                for k in range(j-ticksBefore+1,j+1):
                    checkTotal+=int(openPrices[k])
                checkPrice = checkTotal/ticksBefore


            #If price now is above price we are checking
            if startPrice>checkPrice*(change):

                #Add to tally
                total+=1
                #Set current price bought for
                buyPrice = openPrices[j]

                #Reset time and change so far
                time=0
                returnValue=1

                datapoint=j

                #Iterate until timeout or stop loss or profit taken
                while time < timeout and stopLoss<returnValue<takeProfit:
                    time = datapoint-j
                    currentPrice=openPrices[datapoint]
                    returnValue = currentPrice/buyPrice
                    datapoint+=1
                totalReturn*=returnValue
                tradePercent+=returnValue-fee
        
    try:
        tradePercent=tradePercent/total
    except:
        pass
    #Return the average trade percent, the total number of times, and the parameters
    returnList=[tradePercent, total, paramsList[0], paramsList[1], paramsList[2], paramsList[3], paramsList[4], paramsList[5], paramsList[6], totalReturn]
    return returnList
    # print(tradePercent)
    # print(tradePerDay)

#algo with all the dataframes already passed through
partialAlgo = partial(algo, openPrices=openPrices, closePrices=closePrices, timestamps=timestamps)

In [47]:
#Params
#change
minimum=1
maximum=1.03
difference=0.005
changeList=np.arange(minimum,maximum,difference)

#takeProfit
minimum=1.005
maximum=1.05
difference=0.005
takeProfitList=np.arange(minimum,maximum,difference)

#stopLoss
minimum=0.95
maximum=0.99
difference=0.005
stopLossList=np.arange(minimum,maximum,difference)

#timeout
timeoutList=[5,10,20,30,50]

#ema sma price before
averageTicksBeforeList=[3,5,10,20,50]

#days before list
priceTicksBeforeList=[1,2,3,5,10,20,50]

#minimum total for algo
minTotal=1000

#rough commission fee
fee=[0.00]

#Every combination of parameters
priceCombinations=list(product(changeList, takeProfitList, stopLossList, timeoutList, fee, priceTicksBeforeList, [1]))
emaCombinations=list(product(changeList, takeProfitList, stopLossList, timeoutList, fee, averageTicksBeforeList, [2]))
smaCombinations=list(product(changeList, takeProfitList, stopLossList, timeoutList, fee, averageTicksBeforeList, [3]))
#fullCombinations=priceCombinations+smaCombinations
fullCombinations=priceCombinations+smaCombinations+emaCombinations
len(fullCombinations)

53550

In [48]:
#Test algo to compile it
algo(fullCombinations[0], openPrices, closePrices, timestamps)
#fullCombinations[0]

[0.9999833717394154,
 165048.0,
 1.0,
 1.005,
 0.95,
 5.0,
 0.0,
 1.0,
 1.0,
 0.052587039144521446]

In [50]:
#Apply the algo over the full combinations list with multiprocessing

#List of results to store in right order
results=[]
totalResults=[]
orderCombinations=[]

#Start multiprocessing
with multiprocessing.Pool(40) as pool:
    for result in pool.map(
        partialAlgo,
        fullCombinations
    ):
        #Check if the total tally is high enough to be reasonable
        if result[1]>=minTotal:
            results.append(result[0])
            totalResults.append(result[9])
            orderCombinations.append((result[2],result[3],result[4],int(result[5]),result[6],int(result[7]),result[8]))

In [52]:
#Sort results with largest first, sorts the combinations by the same order
sortedResults, resultCombinations = zip(*sorted(zip(results, orderCombinations), reverse=True))
sortedTotalResults, totalResultCombinations = zip(*sorted(zip(totalResults, orderCombinations), reverse=True))
print(partialAlgo(resultCombinations[0]))
print(partialAlgo(totalResultCombinations[0]))

[1.0012317369713106, 1092.0, 1.005, 1.0349999999999993, 0.96, 50.0, 0.0, 3.0, 1.0, 3.48073725880372]
[1.000056824150019, 334297.0, 1.0, 1.005, 0.95, 50.0, 0.0, 3.0, 3.0, 10024106.634784477]


In [53]:
#Print 10 best return parameter lists
used=[]
j=0
for k in range(10):
    tempResult=partialAlgo(resultCombinations[j])
    while tempResult[1] in used:
        j+=1
        tempResult=partialAlgo(resultCombinations[j])
    used.append(tempResult[1])
    print(tempResult)
    j+=1

#Print 10 best total return parameter lists
used=[]
j=0
for k in range(10):
    tempResult=partialAlgo(totalResultCombinations[j])
    while tempResult[9] in used:
        j+=1
        tempResult=partialAlgo(resultCombinations[j])
    used.append(tempResult[1])
    print(tempResult)
    j+=1

[1.0012317369713106, 1092.0, 1.005, 1.0349999999999993, 0.96, 50.0, 0.0, 3.0, 1.0, 3.48073725880372]
[1.000640483125993, 2131.0, 1.0099999999999998, 1.0249999999999995, 0.955, 30.0, 0.0, 20.0, 1.0, 3.5811121485880806]
[1.0005780448652186, 2309.0, 1.005, 1.0249999999999995, 0.96, 50.0, 0.0, 5.0, 1.0, 3.286492963741601]
[1.0003551721206845, 3280.0, 1.0199999999999996, 1.0299999999999994, 0.96, 50.0, 0.0, 50.0, 3.0, 2.9093989255544828]
[1.0001825194907026, 3156.0, 1.0199999999999996, 1.0349999999999993, 0.96, 50.0, 0.0, 20.0, 3.0, 1.6713853351093269]
[1.00015391367943, 5953.0, 1.005, 1.0149999999999997, 0.955, 20.0, 0.0, 10.0, 1.0, 2.24209096197788]
[1.0000854785030515, 3665.0, 1.0199999999999996, 1.005, 0.975, 30.0, 0.0, 5.0, 3.0, 1.3365452933022501]
[1.000083396672071, 3241.0, 1.0199999999999996, 1.0149999999999997, 0.97, 50.0, 0.0, 20.0, 2.0, 1.2540644146262476]
[1.0000813007950338, 3383.0, 1.0199999999999996, 1.0149999999999997, 0.97, 50.0, 0.0, 10.0, 2.0, 1.2587847922838358]
[1.00007

In [38]:
#return, number of times, change, take profit, stop loss, timeout, fee, ticks before, version, total return

In [55]:
print(partialAlgo(resultCombinations[0]))
print(partialAlgo(totalResultCombinations[0]))

[1.0012317369713106, 1092.0, 1.005, 1.0349999999999993, 0.96, 50.0, 0.0, 3.0, 1.0, 3.48073725880372]
[1.000056824150019, 334297.0, 1.0, 1.005, 0.95, 50.0, 0.0, 3.0, 3.0, 10024106.634784477]


In [56]:
len(results)

29250